In [1]:
#!pip install keras (online GPU)


In [3]:
import pandas as pd
import numpy as np 
import tensorflow

def replaceLabel(value):
    if value == -1:
        return 0
    else:
        return 1

label_cols = ['image_file', 'popularity'] + 'partial_faces is_female baby child teenager youth middle_age senior white black asian oval_face round_face heart_face smiling mouth_open frowning wearing_glasses wearing_sunglasses wearing_lipstick tongue_out duck_face black_hair blond_hair brown_hair red_hair curly_hair straight_hair braid_hair showing_cellphone using_earphone using_mirror braces wearing_hat harsh_lighting dim_lighting'.split(' ')
raw_labels = pd.read_csv('selfie_dataset.txt', delim_whitespace=True, names=label_cols)

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:

num_images = 2000 # len(all_images)
image_size = 256

expression_cols = ['smiling', 'mouth_open', 'frowning', 'tongue_out', 'duck_face']

all_images_x = np.zeros((num_images, image_size, image_size, 3))
all_images_y = np.zeros((num_images, len(expression_cols)))

for i in range(num_images):
    row = raw_labels.iloc[i]
    
    image_file = '/datasets/selfie/images/{}.jpg'.format(row['image_file'])
    
    img = image.load_img(image_file, grayscale=False, target_size=(image_size, image_size), interpolation='nearest')
    img_arr = image.img_to_array(img, data_format='channels_last')
    
    all_images_x[i] = img_arr
    all_images_y[i] = [replaceLabel(raw_labels.iloc[i][x]) for x in expression_cols]



    if 100 * (i+1) // num_images > 100 * i // num_images:
    	print('{}% complete'.format(100 * (i+1) / num_images))

1.0% complete
2.0% complete
3.0% complete
4.0% complete
5.0% complete
6.0% complete
7.0% complete
8.0% complete
9.0% complete
10.0% complete
11.0% complete
12.0% complete
13.0% complete
14.0% complete
15.0% complete
16.0% complete
17.0% complete
18.0% complete
19.0% complete
20.0% complete
21.0% complete
22.0% complete
23.0% complete
24.0% complete
25.0% complete
26.0% complete
27.0% complete
28.0% complete
29.0% complete
30.0% complete
31.0% complete
32.0% complete
33.0% complete
34.0% complete
35.0% complete
36.0% complete
37.0% complete
38.0% complete
39.0% complete
40.0% complete
41.0% complete
42.0% complete
43.0% complete
44.0% complete
45.0% complete
46.0% complete
47.0% complete
48.0% complete
49.0% complete
50.0% complete
51.0% complete
52.0% complete
53.0% complete
54.0% complete
55.0% complete
56.0% complete
57.0% complete
58.0% complete
59.0% complete
60.0% complete
61.0% complete
62.0% complete
63.0% complete
64.0% complete
65.0% complete
66.0% complete
67.0% complete
68.0

In [4]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

datagen.fit(all_images_x)

data_train_flow = datagen.flow(all_images_x, all_images_y, batch_size=64, shuffle=True, subset='training')
data_test_flow = datagen.flow(all_images_x, all_images_y, batch_size=64, shuffle=True, subset='validation')



In [5]:
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
import matplotlib.pyplot as plt

img = image.load_img('/datasets/selfie/images/924956_1438435846397127_1058716982_a.jpg', grayscale=False, target_size=(image_size, image_size), interpolation='nearest')
img_arr = image.img_to_array(img, data_format='channels_last').astype(np.float32) / 255

model = Sequential([
    Conv2D(64, (5, 5), activation='relu', input_shape=(image_size,image_size,3)),
    MaxPooling2D(pool_size=(5,5), strides=(2, 2)),
    
    Conv2D(128, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(5,5), strides=(2, 2)),
    
    Conv2D(256, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(5,5), strides=(2, 2)),
    
    Conv2D(512, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(5,5), strides=(2, 2)),
    
    Conv2D(1024, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(5,5), strides=(2, 2)),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.2),
    
    Dense(len(expression_cols), activation='sigmoid'),
])

model.summary()
import keras.optimizers
model.compile(
    loss='mean_squared_error', 
    optimizer=keras.optimizers.Adam(.10), 
    metrics=['accuracy']
)
history = model.fit_generator(data_train_flow, steps_per_epoch=10, epochs=10)
plt.plot(history.history['acc'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 253, 253, 64)      3136      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 125, 125, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 122, 128)     131200    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 59, 59, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 55, 55, 256)       819456    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 256)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 22, 512)       3277312   
__________